In [246]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dataset
from bs4 import BeautifulSoup 
import requests
from urllib.parse import unquote
from urllib.parse import urljoin

Get to the Network page in the profile 

Scrape the links of the network of professionals connected to you

Build the links into full links to reach their profiles without sign-in

Build Profile_scraper() to get the details inside the profile. Build the function in a way that, much of the code can be re-used for other kind of profile scrapings, 
    Blockchain NFT, Currencies, Chains, Brides / grooms information... to mention some examples

Build dataset of connections' connections. This is where the entire linked-in site can be easily scraped, if the users allow to see their connections...

Without sign-in we can only see the links who are in someway similar to person opened through the link. These may not be connected to your connection. In short the data, coming out of the profile is much limited, without sign-in
Data that can be scraped is listed 

After working on the data from linked-in for for 4 hours, there are couple of points to remember.

    always get the sources of atleast 50 pages and try out to find all the types of errors that can occur.

    list the error observation and find out what kind of x_path mismatch might create the error

    find alternate solution to these errors

    Finally when running the details for the entire list of connections be sure to again, get the source and filter through it

Work on the next step to scrape the messari IO website.... 

learn more about the usage

driver.execute_script("window.scrollBy(0, document.body.scrollHeight-400)")

In [247]:
#Initiating the Selenium Drivers
from selenium import webdriver
import time as time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.chrome.options import Options
import warnings
warnings.filterwarnings('ignore')

In [248]:
entry = 'https://www.linkedin.com'

In [249]:
driver = webdriver.Chrome()
driver.get(entry)
driver.maximize_window()
driver.find_element_by_class_name('nav__button-secondary').click()
driver.find_element_by_id('username').send_keys('kamaljp@gmail.com')
driver.find_element_by_id('password').send_keys('SepdL!n7')
driver.find_element_by_class_name('login__form_action_container').click()
driver.find_element_by_id('ember19').click()

In [250]:
conn_tot = driver.find_element_by_class_name('pl3').text

In [251]:
driver.find_element_by_class_name('pl3').click()

In [252]:
conn_tot = int(conn_tot)
tot_scroll = int(np.floor(conn_tot/10))
tot_scroll

95

In [24]:
for i in range(1,tot_scroll):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    #print(i)
    if i==4:
        driver.execute_script("window.scrollTo(document.body.scrollHeight,window.innerHeight);")
    else:
        continue
html_source = driver.page_source
soup_net = BeautifulSoup(html_source,'html.parser')
driver.quit()

In [81]:
# Don't execute, the program tries to load the cookies from a different session. Errors out
import json

import pickle

browser_cookie = driver.get_cookies()
pickle.dump(driver.get_cookies(),open("cookies_1.pkl","wb"))

with open('cookies', 'w') as j_out:
    json.dump(browser_cookie, j_out)
with open('cookies') as j:
    saved_cookie = json.load(j)
#print(saved_cookie) # The whole cookie as a list with dictionaries 
print(saved_cookie[0]['domain'])
print(saved_cookie[0]['httpOnly'])

#new driver
driver1 = webdriver.Chrome()

# loading the stored cookies
for cookie in saved_cookie:
    driver1.add_cookie(cookie)

.linkedin.com
False


In [93]:
#Don't execute, only reference
t1 = soup_net.find_all('li',attrs={'class':'mn-connection-card artdeco-list'})

In [113]:
#Don't execute, only reference
t1[0].select('a')[1].get('href')

'/in/dhivya-perumal-a0198b1b/'

In [114]:
#Don't execute, only reference
t1[0].select('span')[2].get_text().strip()

'Dhivya Perumal'

In [115]:
#Don't execute, only reference
t1[0].select('span')[4].get_text().strip()

'Assistant Consultant at Tata Consultancy Services'

In [253]:
def get_connLevel_data(beauty_soup, df):
    trial_L1 = beauty_soup.find_all('li',attrs={'class':'mn-connection-card artdeco-list'})
    start = len(df)
    for index,trial in enumerate(trial_L1):
        df.loc[start+index,'Name'] = trial.select('span')[2].get_text().strip()
        df.loc[start+index,'Occupation'] = trial.select('span')[4].get_text().strip()
        df.loc[start+index,'Link'] = urljoin(entry,trial.select('a')[1].get('href'))
    end = len(df)
    print('Total links got is {}'.format(end-start))

In [25]:
data_conn_linkedin = pd.DataFrame()
get_connLevel_data(soup_net,data_conn_linkedin)
data_conn_linkedin.head()
data_conn_linkedin.to_csv('linked_connection.csv')

Total links got is 947


In [254]:
data_conn_linkedin = pd.read_csv('linked_connection.csv')

In [256]:
data_conn_linkedin.drop('Unnamed: 0',axis=1,inplace=True)
data_conn_linkedin.head()

,Name,Occupation,Link
0,Dhivya Perumal,Assistant Consultant at Tata Consultancy Services,https://www.linkedin.com/in/dhivya-perumal-a01...
1,Lakshmana Kumar,Assistant Professor at Sona College of Technology,https://www.linkedin.com/in/lakshmana-kumar-9b...
2,karthikeyan Balasubramani,Deputy Delivery Manager [SDS2/Tekla] at TRC Wo...,https://www.linkedin.com/in/karthikeyan-balasu...
3,Hitansha Bhatia (she/her),Pastry Chef | French Tutor | Psychology Enthus...,https://www.linkedin.com/in/hitansha-bhatia-sh...
4,Prakhar Chauhan (PC),Building state-of-the-art in patients safety a...,https://www.linkedin.com/in/prakhar-chauhan-pc...


In [257]:
#Creating a subroutine to get page
def get_page(main_driver):
    soup = main_driver.page_source
    ready_soup = BeautifulSoup(soup,'html.parser')
    return ready_soup

In [85]:
# Open a new window, and try to use the same browser but sign-out. NOT WORKS
driver.execute_script("window.open('');")
# Switch to the new window
driver.switch_to.window(driver.window_handles[1])
driver.get(page_url)
time.sleep(3)

In [88]:
profile_page = get_page(driver)

In [97]:
#getting total connections
profile_page.find_all('li',attrs={'class':'text-body-small'})[0].select('span')[1].get_text()

'46'

In [104]:
#getting loctions
profile_page.find_all('div',attrs={'class':'mt2 relative'})[0].select('span')[3].get_text().strip()

'Charlotte, North Carolina, United States'

In [110]:
#getting About
profile_page.find_all('div',attrs={'class':'display-flex ph5 pv3'})[0].select('span')[2].get_text()

'12 years of mainframe experience in Banking and Healthcare domain.\nHave very good experenice in Hogan -CIS application '

In [231]:
#getting Mutual Connection
profile_page.find_all('span',attrs={'class':'t-normal t-black--light t-14 hoverable-link-text'})[0].get_text().strip()[:2]

'15'

In [123]:
#getting Mutual Connection
profile_page.find('a',attrs={'class':'app-aware-link display-flex pv1 align-items-center link-without-hover-visited'}).get('href')

'https://www.linkedin.com/search/results/people/?facetNetwork=%22F%22&facetConnectionOf=%22ACoAAAQ88-EBjk3L7ghJ3-bHzR9UboxdBnzVac4%22&origin=MEMBER_PROFILE_CANNED_SEARCH'

In [212]:
#getting Working Currently
profile_page.find_all('ul',attrs={'class':'pvs-list'})[1].find('span',attrs={'aria-hidden':'true'}).get_text().strip()

'Tata Consultancy Services'

In [136]:
#getting working currently in another way...
profile_page.find_all('div',attrs={'class':'pvs-list__outer-container'})[1].select('span')[1].get_text()

'Tata Consultancy Services'

In [144]:
#getting Tot_Skills from "See all Skills"
profile_page.find_all('div',attrs={'class':'pvs-list__outer-container'})[4].select('span')[-1].get_text().strip()[-8]

'9'

In [ ]:
#getting Education...
profile_page.find_all('div',attrs={'class':'pvs-list__outer-container'})[3].select('span')[1].get_text()

In [258]:
def get_skills(main_driver,tot_skills):
    try:
        main_driver.find_elements_by_class_name('pvs-list__footer-wrapper')[2].click() #Clicking show skills
    except NoSuchElementException as nse:
        print('no skills page')
    skill_soup = get_page(driver)
    skill_set = []
    for skill_span in skill_soup.find_all('span',attrs={'aria-hidden':'true'}):
        skill_set.append(skill_span.get_text().strip())
    #While returning, the tot_skills that was collected earlier is used 
    return skill_set[1:int(tot_skills)+1]

In [278]:
def get_profilevel_data(profile_soup, df,conn_index):
    try:
        df.loc[conn_index,'location'] = profile_soup.find_all('div',attrs={'class':'mt2 relative'})[0].select('span')[3].get_text().strip()
    except (IndexError, ValueError,AttributeError):
        df.loc[conn_index,'location'] = 'null'
    
    try:
        df.loc[conn_index,'MuConn_link'] = profile_soup.find('a',attrs={'class':'app-aware-link display-flex pv1 align-items-center link-without-hover-visited'}).get('href')
    except (IndexError, ValueError,AttributeError):
        df.loc[conn_index,'MuConn_link'] = 'null'
    
    #Seperate try and except is required to catch the error from each assignment
    try:
        df.loc[conn_index,'tot_MuConn'] = profile_soup.find_all('span',attrs={'class':'t-normal t-black--light t-14 hoverable-link-text'})[0].get_text().strip()[:2]
    except (IndexError, ValueError,AttributeError):
        df.loc[conn_index,'tot_MuConn'] = 'null'    
    try:
        df.loc[conn_index,'tot_conn'] = profile_soup.find_all('li',attrs={'class':'text-body-small'})[0].select('span')[1].get_text()
    except (IndexError, ValueError,AttributeError):
        df.loc[conn_index,'tot_conn'] = 'null'
    try:
        df.loc[conn_index,'Working_Currently'] = profile_soup.find_all('ul',attrs={'class':'pvs-list'})[1].find('span',attrs={'aria-hidden':'true'}).get_text().strip() 
    except (IndexError, ValueError,AttributeError):
        df.loc[conn_index,'Working_Currently'] = 'null'
    try:
        df.loc[conn_index,'About'] = profile_soup.find_all('div',attrs={'class':'display-flex ph5 pv3'})[0].select('span')[2].get_text() 
    except (IndexError, ValueError,AttributeError):
        df.loc[conn_index,'About'] = 'null'
    try:
        df.loc[conn_index,'Education'] = profile_soup.find_all('div',attrs={'class':'pvs-list__outer-container'})[3].select('span')[1].get_text()
    except (IndexError, ValueError,AttributeError):
        df.loc[conn_index,'Education'] = 'null'
    try:
        df.loc[conn_index,'tot_skills']= profile_soup.find_all('div',attrs={'class':'pvs-list__outer-container'})[4].select('span')[-1].get_text().strip()
    except (IndexError, ValueError,AttributeError):
        df.loc[conn_index,'tot_skills'] = 'null'

In [264]:
sources=[] #stores the profiles locally
def profile_collector(page_url,conn_index,main_driver,stor_dataframe):
    main_driver.get(page_url) #send the existing driver, don't create new

    main_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #Above code initialises the selenium driver and opens the browser and scrolls to the end and captures all information
    
    firstpage = get_page(main_driver) #gets the first page
    sources.append(firstpage)

In [266]:
stor_dataframe = pd.DataFrame() #Stordataframe is only referring, doesn't alter main_dataframe

for index, link in enumerate(data_conn_linkedin.Link):
    print(index)
    #print(link)
    profile_collector(link,index,main_driver=driver,stor_dataframe=stor_dataframe)

0
1
2
3
4


KeyboardInterrupt: 

In [279]:
stor_dataframe = pd.DataFrame()
for index, source in enumerate(sources):
    print(index)
    get_profilevel_data(source,df=stor_dataframe,conn_index=index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


In [281]:
stor_dataframe.head()

,location,MuConn_link,tot_MuConn,tot_conn,Working_Currently,About,Education,tot_skills
0,"Charlotte, North Carolina, United States",https://www.linkedin.com/search/results/people...,15,47,Tata Consultancy Services,12 years of mainframe experience in Banking an...,Sona College of Technology,See all 9 skills
1,"Salem, Tamil Nadu, India",https://www.linkedin.com/search/results/people...,14,155,Assistant Professor,null,Teaching,Endorse
2,"Mysore, Karnataka, India",https://www.linkedin.com/search/results/people...,34,500+,"TRC Worldwide Engineering, Inc.",null,JainUniversity,Grade: First Class
3,"South Delhi, Delhi, India",https://www.linkedin.com/search/results/people...,7,88,Voice Over Artist,I’m a Pastry Chef with passion for flavor and ...,,Issued Dec 2016 · No Expiration Date
4,Greater Delhi Area,https://www.linkedin.com/search/results/people...,14,500+,null,null,Procurement Lead,3PP procurement


In [282]:
driver.quit()

In [283]:
stor_dataframe.describe()

,location,MuConn_link,tot_MuConn,tot_conn,Working_Currently,About,Education,tot_skills
count,54,54,54,54,54,54,54,54
unique,37,48,19,10,35,17,39,38
top,"Bengaluru, Karnataka, India",null,7,500+,null,null,,
freq,7,3,7,28,16,24,12,10


In [291]:
stor_dataframe[stor_dataframe.tot_skills.str.contains('skills')]

,location,MuConn_link,tot_MuConn,tot_conn,Working_Currently,About,Education,tot_skills
0,"Charlotte, North Carolina, United States",https://www.linkedin.com/search/results/people...,15,47,Tata Consultancy Services,12 years of mainframe experience in Banking an...,Sona College of Technology,See all 9 skills
18,"Indore, Madhya Pradesh, India",https://www.linkedin.com/search/results/people...,3,null,rashmi Saini posted this • 4h,null,Shri Shai Nath Higher Secondry School,See all 5 skills
24,Greater Chennai Area,https://www.linkedin.com/search/results/people...,1,72,Sales Coordinator,null,Email Marketing,See all 8 skills
29,"Chennai, Tamil Nadu, India",https://www.linkedin.com/search/results/people...,27,500+,"Malathi Dileepan, Pursuing CIOP posted this • 22h",null,Periyar Maniammai University,See all 6 skills
50,"Charlotte, North Carolina, United States",https://www.linkedin.com/search/results/people...,15,47,Tata Consultancy Services,12 years of mainframe experience in Banking an...,Sona College of Technology,See all 9 skills


In [297]:
stor_dataframe[stor_dataframe.tot_MuConn == 'null']

,location,MuConn_link,tot_MuConn,tot_conn,Working_Currently,About,Education,tot_skills
7,"Talks about #data, #python, #datascience, #mac...",null,null,null,Data Science And AI Content Writer,I am a data science content writer with small ...,Writing weekly articles for the NVIDIA Develop...,Became the first-ever Kaggle Master from Uzbek...
22,Greater St. Louis,null,null,null,null,null,null,null
40,"Mumbai, Maharashtra, India",null,null,500+,Producer,,• Founding member of the International Youth C...,
